# Module 2 Summative Lab

## Introduction

For today's section, we're going to work on a single big lab to apply everything we've learned in Module 2!

## About This Lab

A quick note before getting started--this lab isn't like other labs you seen so far. This lab is meant to take ~8 hours to complete, so it's much longer and more challenging that the average labs you've seen so far. If you feel like this lab is challenging or that you might be struggling a bit, don't fret--that's by design! With everything we've learned about Web Scraping, APIs, and Databases, the best way to test our knowledge of it is to build something substantial! 

## The Project

In this lab, we're going to make use of everything we've learned about APIs, databases, and Object-Oriented Programming to **_Extract, Transform, and Load_** (or **_ETL_**, for short) some data from a SQL database into a MongoDB Database. 

You'll find a database containing information about soccer teams and the matches they've played in the file `database.sqlite`. For this project, our goal is to get the data we think is important from this SQL database, do some calculations and data transformation, and then store everything in a MongoDB database. 

Let's get into the specifics of this project.

### The Goal

Start by examining the data dictionary for the SQL database we'll be working with, which comes from this [kaggle page](https://www.kaggle.com/laudanum/footballdelphi).  Familiarize yourself with the tables it contains, and what each column means. We'll be using this database to get data on each soccer team, calculate some summary statistics, and then store each in a MongoDB database. 

Upon completion of this lab, each unique team in this dataset should have a record in the MongoDB instance containing the following information:

* The name of the team
* The total number of goals scored by the team during the 2011 season
* The total number of wins the team earned during the 2011 season
* A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly by assigning it to a variable)
* The team's win percentage on days where it was raining during games in the 2011 season. 

#### Getting the Weather Data

Note that for this last calculation, you'll need to figure out if it was raining or not during the game. The database itself does not contain this information, but it does contain the date on which the game was played. For this, you'll need to use the [DarkSky API](https://darksky.net/dev) to get the historical weather data for that day. Note that each game is played in a different location, and this information is not contained in our SQL database. However, the teams in this database are largely german, so go ahead and just use the weather in Berlin, Germany as a proxy for this information. If it was raining in Berlin on the day the game was played, count that as rain game--**_you do not need to try and figure out the actual weather at each game's location, because we don't have that information!_**

#### NOTE: The DarkSky API is limited to 1000 free API calls a day, so be sure to test your model on very small samples. Otherwise, you'll hit the rate limit!

## Project Architecture

Unlike previous labs, this lab is more open-ended, and will require you to make design decisions and plan out your strategy for building a system with this many working parts. However, **_using Object-Oriented Programming is a requirement for this project--you must create at least 2 separate, well structured classes in your solution!_** Although it may seem easier to "just start coding", this is a classic beginner's mistake. Instead, think about separating out the different functionalities you'll need to reach your goal, and then build classes to handle each. For instance, at minimum, you'll need to:

* Query the SQL database
* Calculate summary statistics
* Get the weather data from the DarkSky API
* Load the data into MongoDB

We **_strongly recommend_** you consider creating separate classes for handling at least some of these of these tasks.  Be sure to plan the inputs, outputs, and methods for each class before you begin coding! 

**_NOTE:_** We have provided some empty classes below. You are welcome to delete them and use a different architecture for this project if you so choose.  You do not have to use each of them, they are just there to give you an idea of what you could sorts of classes you may want to consider using.

### Rapid Prototyping and Refactoring

It's totally okay to try to get a task working without using OOP. For instance, when experimenting with the DarkSky API for getting historical weather data, it makes sense to just write the code in the cells and rapidly iterate until you get it all working. However, once you get it working, you're not done--you should then **_Refactor_** your code into functions or classes to make your code more modular, reusable, understandable, and maintainable! 

In short--do what you need to do to get each separate piece of functionality working, and then refactor it into a class after you've figured it out!

### Some Final Advice

You haven't built anything this big or complex thus far, so you may not yet fully realize how much trial and error goes into it. If your code keeps breaking, resist the urge to get frustrated, and just keep working. Software development is an iterative process!  No one writes perfect code that works the first time for something this involved. You're going to run into _a lot_ of small errors in this project, right up until the point where it just works, and then you're done! However, you can reduce these errors by planning out your code, and thinking about how all of the pieces fit together before you begin coding. Once you have some basic understanding of how it all will work, then you'll know what you need to build, and then all that is left is to build it!

In short:

* Plan ahead--you'll thank yourself later!
* Errors and broken code aren't bad, they're normal. 
* Keep working, and stay confident--you can do this!

Good luck--we look forward to seeing your completed project!

In [1]:
# You don't have to use these classes, but we recommend them as a good place to start!
class WeatherGetter():
    pass

In [2]:
class MongoHandler():
    pass

In [3]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('database.sqlite')
cur = conn.cursor()

In [4]:
cur.execute("""SELECT * FROM matches;""") #24625 rows, 9 columns
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A


In [5]:
cur.execute("""SELECT * FROM teams;""") # 468 rows, 8 columns
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
#print(df.info())
df.head(10)

,Season,TeamName,KaderHome,AvgAgeHome,ForeignPlayersHome,OverallMarketValueHome,AvgMarketValueHome,StadiumCapacity
0,2017,Bayern Munich,27,26,15,597950000,22150000,75000
1,2017,Dortmund,33,25,18,416730000,12630000,81359
2,2017,Leverkusen,31,24,15,222600000,7180000,30210
3,2017,RB Leipzig,30,23,15,180130000,6000000,42959
4,2017,Schalke 04,29,24,17,179550000,6190000,62271
5,2017,M'gladbach,31,25,17,154400000,4980000,54014
6,2017,Wolfsburg,31,24,14,124430000,4010000,30000
7,2017,FC Koln,24,26,9,118550000,4940000,49968
8,2017,Hoffenheim,31,24,14,107330000,3460000,30164
9,2017,Hertha,26,26,12,86800000,3340000,74475


In [6]:
cur.execute("""SELECT * FROM teams_in_matches;""") #49148 entries (rows), 2 columns
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()


,Match_ID,Unique_Team_ID
0,1,26
1,1,46
2,2,26
3,2,42
4,3,26


In [7]:
cur.execute("""SELECT * FROM unique_teams limit 5;""")#128 teams (rows), 2 columns
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,TeamName,Unique_Team_ID
0,Bayern Munich,1
1,Dortmund,2
2,Leverkusen,3
3,RB Leipzig,4
4,Schalke 04,5


In [8]:
# cur.execute("""SELECT * FROM sqlite_sequence;""") #2 rows, 2 columns, don't know value of this table
# df = pd.DataFrame(cur.fetchall())
# df.columns = [x[0] for x in cur.description]
# df

In [9]:
cur.execute('''SELECT TeamName FROM teams WHERE season = '2011';''')
df_teams = pd.DataFrame(cur.fetchall())
# cur.execute('''SELECT count(TeamName) FROM teams WHERE season = '2011';''').fetchall() # There are 36 teams
df_teams.columns = ['team']
df_teams

,team
0,Bayern Munich
1,Dortmund
2,Leverkusen
3,Schalke 04
4,Werder Bremen
5,Wolfsburg
6,Hamburg
7,Stuttgart
8,Hoffenheim
9,FC Koln


In [12]:
#using df to pass in each name separatly
# for row in df_teams:
    
#     name = df_teams[row]

#     cur.execute('''SELECT SUM(FTHG) as total_goals FROM matches 
#                WHERE homeTeam = ?
#                AND Date BETWEEN '2011-01-01' AND '2011-12-31',(name,);''')
#     df_home_goals = pd.DataFrame(cur.fetchall())
#     df_home_goals.columns = [x[0] for x in cur.description]

#     cur.execute('''SELECT SUM(FTAG) as total_goals FROM matches 
#                WHERE awayTeam = ?
#                AND Date BETWEEN '2011-01-01' AND '2011-12-31',(name,);''')
#     df_away_goals = pd.DataFrame(cur.fetchall())
#     df_away_goals.columns = [x[0] for x in cur.description]

#     tot_goals = df_home_goals + df_away_goals
    
#     df_teams['total_goals'] = tot_goals
# df_teams

In [15]:
#second attempt using df to pass in each name separatly

for row in df_teams:
    
    name = df_teams[row]

    cur.execute("SELECT SUM(FTHG) as total_goals FROM matches 
               WHERE homeTeam = '%s'
               AND Date BETWEEN '2011-01-01' AND '2011-12-31';" % (name))
    df_home_goals = pd.DataFrame(cur.fetchall())
    df_home_goals.columns = [x[0] for x in cur.description]

    cur.execute('''SELECT SUM(FTAG) as total_goals FROM matches 
               WHERE awayTeam = '%s'
               AND Date BETWEEN '2011-01-01' AND '2011-12-31';''', (name))
    df_away_goals = pd.DataFrame(cur.fetchall())
    df_away_goals.columns = [x[0] for x in cur.description]

    tot_goals = df_home_goals + df_away_goals
    df_teams['total_goals'] = tot_goals

df_teams

OperationalError: near "gladbach": syntax error

In [17]:
# total goals for one team

cur.execute('''SELECT SUM(FTHG) as total_goals FROM matches 
               WHERE ((homeTeam = 'Bayern Munich'))
               AND Date BETWEEN '2011-01-01' AND '2011-12-31';''')
df_home_goals = pd.DataFrame(cur.fetchall())
df_home_goals.columns = [x[0] for x in cur.description]

cur.execute('''SELECT SUM(FTAG) as total_goals FROM matches 
               WHERE ((awayTeam = 'Bayern Munich'))
               AND Date BETWEEN '2011-01-01' AND '2011-12-31';''')
df_away_goals = pd.DataFrame(cur.fetchall())
df_away_goals.columns = [x[0] for x in cur.description]

tot_goals = df_home_goals + df_away_goals
print(tot_goals)


   total_goals
0           93


In [18]:
# duplicate of above to try subqueries with all teams

cur.execute('''SELECT SUM(FTHG) as total_goals FROM matches 
               WHERE Date BETWEEN '2011-01-01' AND '2011-12-31'
               AND homeTeam IN (SELECT TeamName FROM teams WHERE season = '2011')
                                ;''')
df_hg = pd.DataFrame(cur.fetchall())
df_hg.columns = [x[0] for x in cur.description]

cur.execute('''SELECT SUM(FTAG) as total_goals FROM matches 
               WHERE Date BETWEEN '2011-01-01' AND '2011-12-31'
               AND awayTeam IN (SELECT TeamName FROM teams WHERE season = '2011')
                               ;''')
df_ag = pd.DataFrame(cur.fetchall())
df_ag.columns = [x[0] for x in cur.description]

tot_g = df_hg + df_ag
print(tot_g)


   total_goals
0         1733


In [19]:
# statistics of total wins for one team
cur.execute('''SELECT COUNT(FTR) as total_wins FROM matches 
               WHERE (homeTeam = 'Bayern Munich' AND FTR = 'H')
               AND Date BETWEEN '2011-01-01' AND '2011-12-31';''')
df_home_win = pd.DataFrame(cur.fetchall())
df_home_win.columns = [x[0] for x in cur.description]

cur.execute('''SELECT COUNT(FTR) as total_wins FROM matches 
               WHERE (awayTeam = 'Bayern Munich' AND FTR = 'A')
               AND Date BETWEEN '2011-01-01' AND '2011-12-31';''')
df_away_win = pd.DataFrame(cur.fetchall())
df_away_win.columns = [x[0] for x in cur.description]

tot_win = df_home_win + df_away_win
print(tot_win)
# df_total_stats['total_win'] = tot_win
# print(df_total_stats)

   total_wins
0          23


In [20]:
# statistics of total losses for a team 

# changed Date BETWEEN '2011-01-01' AND '2011-12-31' to season = '2011'

cur.execute('''SELECT COUNT(FTR) as total_loss FROM matches 
               WHERE (homeTeam = 'Bayern Munich' AND FTR != 'H')
               AND Season = '2011';''')
df_home_loss = pd.DataFrame(cur.fetchall())
df_home_loss.columns = [x[0] for x in cur.description]

cur.execute('''SELECT COUNT(FTR) as total_loss FROM matches 
               WHERE (awayTeam = 'Bayern Munich' AND FTR != 'A')
               AND Season = '2011';''')
df_away_loss = pd.DataFrame(cur.fetchall())
df_away_loss.columns = [x[0] for x in cur.description]

tot_loss = df_home_loss + df_away_loss
print(tot_loss)

# df_total_stats['total_loss'] = tot_loss
# print(df_total_stats)

   total_loss
0          11


In [21]:
df_teams['total_loss'] = tot_loss
df_teams['total_win'] = tot_win
df_teams['total_goals'] = tot_goals
print(df_teams)

                  team  total_loss  total_win  total_goals
0        Bayern Munich        11.0       23.0         93.0
1             Dortmund         NaN        NaN          NaN
2           Leverkusen         NaN        NaN          NaN
3           Schalke 04         NaN        NaN          NaN
4        Werder Bremen         NaN        NaN          NaN
5            Wolfsburg         NaN        NaN          NaN
6              Hamburg         NaN        NaN          NaN
7            Stuttgart         NaN        NaN          NaN
8           Hoffenheim         NaN        NaN          NaN
9              FC Koln         NaN        NaN          NaN
10            Hannover         NaN        NaN          NaN
11          M'gladbach         NaN        NaN          NaN
12            Freiburg         NaN        NaN          NaN
13               Mainz         NaN        NaN          NaN
14              Hertha         NaN        NaN          NaN
15      Kaiserslautern         NaN        NaN          N

In [22]:
# dates of wins for a team, pass to Dark Sky, figure out how many games had rain

cur.execute('''SELECT date FROM matches 
               WHERE ((homeTeam = 'Bayern Munich' AND FTR = 'H') OR (awayTeam = 'Bayern Munich' AND FTR = 'A'))
               AND Date BETWEEN '2011-01-01' AND '2011-12-31';''')
date_win = pd.DataFrame(cur.fetchall())
date_win.columns = ['date']
print(date_win)

          date
0   2011-01-29
1   2011-03-19
2   2011-05-07
3   2011-02-19
4   2011-03-12
5   2011-01-22
6   2011-04-17
7   2011-04-30
8   2011-05-14
9   2011-04-02
10  2011-02-12
11  2011-12-11
12  2011-08-13
13  2011-09-18
14  2011-08-27
15  2011-11-06
16  2011-12-16
17  2011-09-10
18  2011-08-20
19  2011-09-24
20  2011-10-29
21  2011-12-03
22  2011-10-15


In [26]:
import time
import datetime
import requests

s = "2011-01-01"
dt = round(time.mktime(datetime.datetime.strptime(s, "%Y-%m-%d").timetuple()))
dt = str(dt)
#print(dt)
resp = requests.get('https://api.darksky.net/forecast/937b41dee3e997c0548c743ff4a79c20/52.52,13.4050,' + dt)
data=resp.json()
print(data['currently']['summary'])


Possible Drizzle


In [27]:
from datetime import datetime

date_win['date'] = pd.to_datetime(date_win['date'])
print(date_win)

# for i in range(0,len(date_win)):
    
#     s = date_win.iloc[i]
#     print(s)
    
#     dt = round(time.mktime(datetime.datetime.strptime(s, "%Y-%m-%d").timetuple()))
#     dt = str(dt)
#     print(dt)
#     resp = requests.get('https://api.darksky.net/forecast/937b41dee3e997c0548c743ff4a79c20/52.52,13.4050,' + dt)
#     data=resp.json()
#     print(data['currently']['summary'])



# df = pd.DataFrame(date_win, columns=['dates'])
# df['dates'] = pd.to_datetime(df['dates'])

#dates = df_date_win['date'].dt.strftime('%Y-%m-%d').values

for index,row in date_win.iterrows():
    print(row['date'])
    print('-')
    s = row['date']

    print(s)
    dt = round(time.mktime(datetime.datetime.strptime(s, "%Y-%m-%d").timetuple()))
    print(dt)

         date
0  2011-01-29
1  2011-03-19
2  2011-05-07
3  2011-02-19
4  2011-03-12
5  2011-01-22
6  2011-04-17
7  2011-04-30
8  2011-05-14
9  2011-04-02
10 2011-02-12
11 2011-12-11
12 2011-08-13
13 2011-09-18
14 2011-08-27
15 2011-11-06
16 2011-12-16
17 2011-09-10
18 2011-08-20
19 2011-09-24
20 2011-10-29
21 2011-12-03
22 2011-10-15
2011-01-29 00:00:00
-
2011-01-29 00:00:00


AttributeError: type object 'datetime.datetime' has no attribute 'datetime'

### Project Plan
- Find teams that played in 2011 season, assign to df_teams
- Create a query to find total goals, total wins and total losses for a team in 2011
- Create a "def statistics" for statistics query (see above)
- Create a query to find the date of a win ("win_date")
- Create a "def win_date" for date of win
- Pass "win_date" to class weathergetter to determine if rain or not using Dark Sky api, return 1 for rain otherwise 0
- Create a mongo database (with keys: team_name, total_goals, total_wins, rain_wins, histogram)
- Create class mongohandler to pass in statistics to add to the mongo db
- Create a for loop that will iterate over df_teams
 - get total wins, total losses, total goals for each team
 - use class weathergetter to get 1 or 0 with win_dates to get % of wins
 - use class mongohandler to add statistics to database

# Summary

In this lab, we dug deep and used everything we've learned so far about python programming, databases, HTTP requests and API calls to ETL data from a SQL database into a MongoDB instance!

In [ ]:
# PARKING LOT FOR DARK SKY

#,exclude=[minutely,hourly,daily,alerts,flags]
# import time
# import datetime
# s = "2011-01-01"
# dt = round(time.mktime(datetime.datetime.strptime(s, "%Y-%m-%d").timetuple()))
# print(dt)
# resp = requests.get('https://api.darksky.net/forecast/937b41dee3e997c0548c743ff4a79c20/52.52,13.4050,dt')
# print(resp.text)

#unix time conversion of 03-12-2011 to 1299888000
#resp = requests.get('https://api.darksky.net/forecast/937b41dee3e997c0548c743ff4a79c20/52.52,13.4050,1299888000')
#resp.status_code == requests.codes.ok

#print (resp.text)
#print(dict(resp.headers))

# data=resp.json()
# print(data['currently']['summary'])

# QUERIES
# Dates of wins, to be passed to Dark Sky 
# cur.execute('''SELECT Date FROM matches 
#                JOIN teams
#                ON teams.teamName = matches.homeTeam
#                WHERE teamName = 'Bayern Munich' 
#                AND FTR ='H'
#                AND Date BETWEEN '2011-01-01' AND '2011-12-31';''').fetchall()

# statistics of  team
# cur.execute('''SELECT COUNT(FTR) as total_wins FROM matches 
#                WHERE (homeTeam = 'Bayern Munich' AND FTR = 'H')
#                AND Date BETWEEN '2011-01-01' AND '2011-12-31';''')
# df_home_stats = pd.DataFrame(cur.fetchall())
# df_home_stats.columns = [x[0] for x in cur.description]
# print(df_home_stats)

# cur.execute('''SELECT COUNT(FTR) as total_wins FROM matches 
#                WHERE (awayTeam = 'Bayern Munich' AND FTR = 'A')
#                AND Date BETWEEN '2011-01-01' AND '2011-12-31';''')
# df_away_stats = pd.DataFrame(cur.fetchall())
# df_away_stats.columns = [x[0] for x in cur.description]
# print(df_away_stats)

# df_total_stats = df_home_stats + df_away_stats
# print(df_total_stats)

# cur.execute('''SELECT homeTeam, awayTeam, date, FTHG, FTAG FROM matches 
#                JOIN teams_in_matches
#                USING(match_ID)
#                JOIN unique_teams
#                USING(unique_team_ID)
#                JOIN teams
#                USING(teamName)
#                WHERE teamName = 'Bayern Munich';''').fetchall() 

# cur.execute('''SELECT homeTeam, SUM(FTHG), COUNT(FTR) FROM matches 
#                JOIN teams
#                ON teams.teamName = matches.homeTeam
#                WHERE teamName = 'Bayern Munich' 
#                AND FTR ='H'
#                AND Date BETWEEN '2011-01-01' AND '2011-12-31';''').fetchall() 

# cur.execute('''SELECT count(FTR) FROM matches 
#                JOIN teams
#                ON teams.teamName = matches.homeTeam
#                WHERE teamName = 'Bayern Munich' 
#                AND FTR = 'H'
#                AND Date BETWEEN '2011-01-01' AND '2011-12-31';''').fetchall() 

# cur.execute('''SELECT homeTeam team, SUM(FTHG) totalGoals, COUNT(FTR) totalWins FROM matches 
#                JOIN teams
#                ON teams.teamName = matches.homeTeam
#                WHERE teamName = 'Bayern Munich' 
#                AND FTR ='H'
#                AND Date BETWEEN '2011-01-01' AND '2011-12-31';''').fetchall()

# cur.execute('''SELECT awayTeam team, SUM(FTAG) totalGoals, COUNT(FTR) totalWins FROM matches 
#                JOIN teams
#                ON teams.teamName = matches.awayTeam
#                WHERE teamName = 'Bayern Munich' 
#                AND FTR = 'A'
#                AND Date BETWEEN '2011-01-01' AND '2011-12-31';''').fetchall() 

# cur.execute('''SELECT team, SUM(totalGoals), SUM(totalWins)
#                FROM (
#                        SELECT homeTeam team, SUM(FTHG) totalGoals, COUNT(FTR) totalWins FROM matches 
#                        JOIN teams
#                        ON teams.teamName = matches.homeTeam
#                        WHERE teamName = 'Bayern Munich' 
#                        AND FTR ='H'
#                        AND Date BETWEEN '2011-01-01' AND '2011-12-31'
#                        UNION ALL
#                        SELECT awayTeam team, SUM(FTAG) totalGoals, COUNT(FTR) totalWins FROM matches 
#                        JOIN teams
#                        ON teams.teamName = matches.awayTeam
#                        WHERE teamName = 'Bayern Munich' 
#                        AND FTR = 'A'
#                        AND Date BETWEEN '2011-01-01' AND '2011-12-31'
#                    )
#                 ;''')